# Telugu Sentiment Analysis Using MuRIL

## 1. MuRIL Embeddings with In-Built Classifier on Translated English Comments

In [ ]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm

In [ ]:


# ——— Project-specific hyperparameters ———
MODEL_NAME    = "google/muril-base-cased"  # MODIFIED to MuRIL
MAX_LEN       = 128
BATCH_SIZE    = 32
LEARNING_RATE = 3e-5
WEIGHT_DECAY  = 0.01
EPOCHS        = 10
WARMUP_RATIO  = 0.1
LABEL_MAP     = {"Negative": 0, "Neutral": 1, "Positive": 2}

# ——— 1. Load & preprocess data ———
df = pd.read_excel('/home/santhosh/Data Scraping/scource code/TSA codes/TSAC - Telugu Sentiment Analysis Corpus.xlsx')[['Translated_English', 'Label']]
df = df.dropna().reset_index(drop=True)
df = df[df["Label"].isin(LABEL_MAP)].reset_index(drop=True)
df["Label"] = df["Label"].map(LABEL_MAP)

# ——— 2. Train/test split ———
train_df, test_df = train_test_split(
    df,
    test_size=0.2,
    stratify=df["Label"],
    random_state=42
)

# ——— 3. Dataset ———
class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts     = texts
        self.labels    = labels
        self.tokenizer = tokenizer
        self.max_len   = max_len

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = int(self.labels[idx])
        enc = self.tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=self.max_len,
            return_tensors="pt"
        )
        return {
            "input_ids": enc["input_ids"].squeeze(0),
            "attention_mask": enc["attention_mask"].squeeze(0),
            "labels": torch.tensor(label, dtype=torch.long)
        }

# ——— 4. Tokenizer & DataLoaders ———
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

train_ds = SentimentDataset(train_df["Translated_English"].tolist(), train_df["Label"].tolist(), tokenizer, MAX_LEN)
test_ds  = SentimentDataset(test_df["Translated_English"].tolist(),  test_df["Label"].tolist(),  tokenizer, MAX_LEN)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
test_loader  = DataLoader(test_ds,  batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

# ——— 5. MuRIL with classification head ———
class MuRILClassifier(nn.Module):
    def __init__(self, model_name, n_classes):
        super().__init__()
        self.base = AutoModel.from_pretrained(model_name)
        self.drop = nn.Dropout(0.3)
        hidden_size = self.base.config.hidden_size
        self.fc = nn.Linear(hidden_size, n_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.base(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output  # MuRIL has pooler_output
        x = self.drop(pooled_output)
        return self.fc(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MuRILClassifier(MODEL_NAME, n_classes=3).to(device)

# ——— 6. Optimizer, Scheduler, Loss ———
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
total_steps = len(train_loader) * EPOCHS
warmup_steps = int(WARMUP_RATIO * total_steps)

scheduler = optim.lr_scheduler.LinearLR(optimizer, start_factor=0.1, total_iters=warmup_steps)
loss_fn = nn.CrossEntropyLoss().to(device)

# ——— 7. Training Loop ———
for epoch in range(1, EPOCHS + 1):
    model.train()
    total_loss = 0
    correct = 0

    for batch in tqdm(train_loader, desc=f"Epoch {epoch}"):
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        logits = model(input_ids, attention_mask)
        loss = loss_fn(logits, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()
        preds = logits.argmax(dim=1)
        correct += (preds == labels).sum().item()

    avg_loss = total_loss / len(train_loader)
    acc = correct / len(train_ds)
    print(f"Epoch {epoch}/{EPOCHS} — Loss: {avg_loss:.4f}, Accuracy: {acc:.4f}")

# ——— 8. Evaluation ———
model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        logits = model(input_ids, attention_mask)
        preds = logits.argmax(dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# ——— 9. Metrics ———
print("\nTest Accuracy:", accuracy_score(all_labels, all_preds))
print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=["Negative", "Neutral", "Positive"]))

cm = confusion_matrix(all_labels, all_preds, labels=[0, 1, 2])
print("\nConfusion Matrix:\n", cm)


2025-06-11 13:31:57.931428: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-11 13:31:57.942017: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749628917.954651  717402 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749628917.958281  717402 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749628917.968194  717402 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Epoch 1/10 — Loss: 1.0977, Accuracy: 0.3421


Epoch 2: 100%|██████████| 33/33 [00:07<00:00,  4.44it/s]


Epoch 2/10 — Loss: 1.0685, Accuracy: 0.4781


Epoch 3: 100%|██████████| 33/33 [00:07<00:00,  4.42it/s]


...

Epoch 9: 100%|██████████| 33/33 [00:07<00:00,  4.31it/s]


Epoch 9/10 — Loss: 0.3202, Accuracy: 0.9300


Epoch 10: 100%|██████████| 33/33 [00:07<00:00,  4.28it/s]

Epoch 10/10 — Loss: 0.2857, Accuracy: 0.9339



Test Accuracy: 0.7248062015503876

Classification Report:
              precision    recall  f1-score   support

    Negative       0.67      0.59      0.62        85
     Neutral       0.79      0.74      0.76        85
    Positive       0.72      0.84      0.77        88

    accuracy                           0.72       258
   macro avg       0.72      0.72      0.72       258
weighted avg       0.72      0.72      0.72       258


Confusion Matrix:
 [[50 16 19]
 [12 63 10]
 [13  1 74]]


## 2. MuRIL Embeddings with In-Built Classifier on Original Telugu Comments

In [ ]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm


In [2]:

# ——— Project-specific hyperparameters ———
MODEL_NAME    = "google/muril-base-cased"  # MODIFIED to MuRIL
MAX_LEN       = 128
BATCH_SIZE    = 32
LEARNING_RATE = 3e-5
WEIGHT_DECAY  = 0.01
EPOCHS        = 10
WARMUP_RATIO  = 0.1
LABEL_MAP     = {"Negative": 0, "Neutral": 1, "Positive": 2}

# ——— 1. Load & preprocess data ———
df = pd.read_excel('/home/santhosh/Data Scraping/scource code/TSA codes/TSAC - Telugu Sentiment Analysis Corpus.xlsx')[['Statement', 'Label']]
df = df.dropna().reset_index(drop=True)
df = df[df["Label"].isin(LABEL_MAP)].reset_index(drop=True)
df["Label"] = df["Label"].map(LABEL_MAP)

# ——— 2. Train/test split ———
train_df, test_df = train_test_split(
    df,
    test_size=0.2,
    stratify=df["Label"],
    random_state=42
)

# ——— 3. Dataset ———
class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts     = texts
        self.labels    = labels
        self.tokenizer = tokenizer
        self.max_len   = max_len

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = int(self.labels[idx])
        enc = self.tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=self.max_len,
            return_tensors="pt"
        )
        return {
            "input_ids": enc["input_ids"].squeeze(0),
            "attention_mask": enc["attention_mask"].squeeze(0),
            "labels": torch.tensor(label, dtype=torch.long)
        }

# ——— 4. Tokenizer & DataLoaders ———
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

train_ds = SentimentDataset(train_df["Statement"].tolist(), train_df["Label"].tolist(), tokenizer, MAX_LEN)
test_ds  = SentimentDataset(test_df["Statement"].tolist(),  test_df["Label"].tolist(),  tokenizer, MAX_LEN)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
test_loader  = DataLoader(test_ds,  batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

# ——— 5. MuRIL with classification head ———
class MuRILClassifier(nn.Module):
    def __init__(self, model_name, n_classes):
        super().__init__()
        self.base = AutoModel.from_pretrained(model_name)
        self.drop = nn.Dropout(0.3)
        hidden_size = self.base.config.hidden_size
        self.fc = nn.Linear(hidden_size, n_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.base(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output  # MuRIL has pooler_output
        x = self.drop(pooled_output)
        return self.fc(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MuRILClassifier(MODEL_NAME, n_classes=3).to(device)

# ——— 6. Optimizer, Scheduler, Loss ———
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
total_steps = len(train_loader) * EPOCHS
warmup_steps = int(WARMUP_RATIO * total_steps)

scheduler = optim.lr_scheduler.LinearLR(optimizer, start_factor=0.1, total_iters=warmup_steps)
loss_fn = nn.CrossEntropyLoss().to(device)

# ——— 7. Training Loop ———
for epoch in range(1, EPOCHS + 1):
    model.train()
    total_loss = 0
    correct = 0

    for batch in tqdm(train_loader, desc=f"Epoch {epoch}"):
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        logits = model(input_ids, attention_mask)
        loss = loss_fn(logits, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()
        preds = logits.argmax(dim=1)
        correct += (preds == labels).sum().item()

    avg_loss = total_loss / len(train_loader)
    acc = correct / len(train_ds)
    print(f"Epoch {epoch}/{EPOCHS} — Loss: {avg_loss:.4f}, Accuracy: {acc:.4f}")

# ——— 8. Evaluation ———
model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        logits = model(input_ids, attention_mask)
        preds = logits.argmax(dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# ——— 9. Metrics ———
print("\nTest Accuracy:", accuracy_score(all_labels, all_preds))
print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=["Negative", "Neutral", "Positive"]))

cm = confusion_matrix(all_labels, all_preds, labels=[0, 1, 2])
print("\nConfusion Matrix:\n", cm)


Epoch 1: 100%|██████████| 33/33 [00:07<00:00,  4.38it/s]


Epoch 1/10 — Loss: 1.0976, Accuracy: 0.3285


Epoch 2: 100%|██████████| 33/33 [00:07<00:00,  4.35it/s]


Epoch 2/10 — Loss: 1.0669, Accuracy: 0.5238


Epoch 3: 100%|██████████| 33/33 [00:07<00:00,  4.34it/s]


...

Epoch 9: 100%|██████████| 33/33 [00:07<00:00,  4.20it/s]


Epoch 9/10 — Loss: 0.6870, Accuracy: 0.7133


Epoch 10: 100%|██████████| 33/33 [00:07<00:00,  4.19it/s]

Epoch 10/10 — Loss: 0.6296, Accuracy: 0.7434



Test Accuracy: 0.6434108527131783

Classification Report:
              precision    recall  f1-score   support

    Negative       0.50      0.62      0.56        85
     Neutral       0.73      0.82      0.77        85
    Positive       0.75      0.49      0.59        88

    accuracy                           0.64       258
   macro avg       0.66      0.65      0.64       258
weighted avg       0.66      0.64      0.64       258


Confusion Matrix:
 [[53 21 11]
 [12 70  3]
 [40  5 43]]


## 3. Generate and Save MuRIL Embeddings

In [ ]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from tqdm import tqdm

In [ ]:


# ——— Config ———
MODEL_NAME = "google/muril-base-cased"
MAX_LEN = 128
BATCH_SIZE = 32
LABEL_MAP = {"Negative": 0, "Neutral": 1, "Positive": 2}
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ——— Load Data ———
df = pd.read_excel('/home/santhosh/Data Scraping/scource code/TSA codes/TSAC - Telugu Sentiment Analysis Corpus.xlsx')[['Translated_English', 'Label']]
df = df.dropna().reset_index(drop=True)
df = df[df["Label"].isin(LABEL_MAP)].reset_index(drop=True)
df["Label"] = df["Label"].map(LABEL_MAP)

# ——— Dataset Class ———
class SentimentDataset(Dataset):
    def __init__(self, texts, tokenizer, max_len):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        enc = self.tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=self.max_len,
            return_tensors="pt"
        )
        return {
            "input_ids": enc["input_ids"].squeeze(0),
            "attention_mask": enc["attention_mask"].squeeze(0)
        }

# ——— Load Tokenizer & Model ———
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME).to(device)
model.eval()

# ——— Embedding Extraction ———
def extract_embeddings(texts):
    dataset = SentimentDataset(texts, tokenizer, MAX_LEN)
    loader = DataLoader(dataset, batch_size=BATCH_SIZE)

    embeddings = []
    with torch.no_grad():
        for batch in tqdm(loader, desc="Extracting Embeddings"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            cls_embeddings = outputs.last_hidden_state[:, 0, :]  # CLS token
            embeddings.append(cls_embeddings.cpu())
    return torch.cat(embeddings).numpy()

# ——— Generate and Save All Embeddings + Labels ———
all_embeddings = extract_embeddings(df["Translated_English"].tolist())
df_embeddings = pd.DataFrame(all_embeddings)
df_embeddings["label"] = df["Label"].values
df_embeddings.to_csv("muril_embeddings.csv", index=False)

## 4. Load Saved Embeddings and Apply Traditional ML Classifiers on Translated English Comments

In [6]:
# ——— Load embeddings from CSV file ———
csv_path = "/home/santhosh/Data Scraping/scource code/muril_embeddings.csv"
df_embeddings = pd.read_csv(csv_path)

# ——— Train-Test Split ———
X = df_embeddings.drop("label", axis=1).values
y = df_embeddings["label"].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# ——— Feature Scaling ———
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# ——— Define Models and Parameter Grids ———
models_params = {
    "LogisticRegression": (
        LogisticRegression(max_iter=1000),
        {"C": [0.1, 1, 10]}
    ),
    "SVM": (
        SVC(),
        {"C": [0.1, 1, 10], "kernel": ["linear", "rbf"]}
    ),
    "RandomForest": (
        RandomForestClassifier(),
        {"n_estimators": [100, 200], "max_depth": [None, 10, 20]}
    ),
    "KNN": (
        KNeighborsClassifier(),
        {"n_neighbors": [3, 5, 7]}
    ),
    "XGBoost": (
        XGBClassifier(use_label_encoder=False, eval_metric="mlogloss"),
        {"n_estimators": [100, 200], "max_depth": [3, 5], "learning_rate": [0.05, 0.1]}
    )
}

# ——— Train + Evaluate Each Model ———
for name, (model, param_grid) in models_params.items():
    print(f"\n🔍 Running GridSearch for {name}...")
    grid = GridSearchCV(model, param_grid, cv=3, scoring="accuracy", n_jobs=-1)
    grid.fit(X_train, y_train)
    y_pred = grid.predict(X_test)

    print(f"\n✅ Best Parameters for {name}: {grid.best_params_}")
    print(f"📊 Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    print("📑 Classification Report:\n", classification_report(y_test, y_pred, target_names=["Negative", "Neutral", "Positive"]))
    print("🔢 Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


🔍 Running GridSearch for LogisticRegression...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 


✅ Best Parameters for LogisticRegression: {'C': 0.1}
📊 Accuracy: 0.6705
📑 Classification Report:
               precision    recall  f1-score   support

    Negative       0.58      0.62      0.60        85
     Neutral       0.75      0.73      0.74        85
    Positive       0.70      0.66      0.68        88

    accuracy                           0.67       258
   macro avg       0.67      0.67      0.67       258
weighted avg       0.67      0.67      0.67       258

🔢 Confusion Matrix:
 [[53 12 20]
 [18 62  5]
 [21  9 58]]

🔍 Running GridSearch for SVM...

✅ Best Parameters for SVM: {'C': 1, 'kernel': 'rbf'}
📊 Accuracy: 0.7209
📑 Classification Report:
               precision    recall  f1-score   support

    Negative       0.58      0.75      0.66        85
     Neutral       0.90      0.72      0.80        85
    Positive       0.76      0.69      0.73        88

    accuracy                           0.72       258
   macro avg       0.75      0.72      0.73       258
weig

/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [11:39:32] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [11:39:33] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [11:39:33] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [11:39:37] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [11:39:37] WARNING: /workspace/src/learner.cc:740: 
Parame


✅ Best Parameters for XGBoost: {'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 200}
📊 Accuracy: 0.6667
📑 Classification Report:
               precision    recall  f1-score   support

    Negative       0.56      0.59      0.57        85
     Neutral       0.74      0.76      0.75        85
    Positive       0.71      0.65      0.68        88

    accuracy                           0.67       258
   macro avg       0.67      0.67      0.67       258
weighted avg       0.67      0.67      0.67       258

🔢 Confusion Matrix:
 [[50 19 16]
 [13 65  7]
 [27  4 57]]


## 5. Load Saved Embeddings and Apply Traditional ML Classifiers on Original Telugu Comments


In [7]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from tqdm import tqdm

# ——— Config ———
MODEL_NAME = "google/muril-base-cased"
MAX_LEN = 128
BATCH_SIZE = 32
LABEL_MAP = {"Negative": 0, "Neutral": 1, "Positive": 2}
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ——— Load Data ———
df = pd.read_excel('/home/santhosh/Data Scraping/scource code/TSA codes/TSAC - Telugu Sentiment Analysis Corpus.xlsx')[['Statement', 'Label']]
df = df.dropna().reset_index(drop=True)
df = df[df["Label"].isin(LABEL_MAP)].reset_index(drop=True)
df["Label"] = df["Label"].map(LABEL_MAP)

# ——— Dataset Class ———
class SentimentDataset(Dataset):
    def __init__(self, texts, tokenizer, max_len):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        enc = self.tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=self.max_len,
            return_tensors="pt"
        )
        return {
            "input_ids": enc["input_ids"].squeeze(0),
            "attention_mask": enc["attention_mask"].squeeze(0)
        }

# ——— Load Tokenizer & Model ———
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME).to(device)
model.eval()

# ——— Embedding Extraction ———
def extract_embeddings(texts):
    dataset = SentimentDataset(texts, tokenizer, MAX_LEN)
    loader = DataLoader(dataset, batch_size=BATCH_SIZE)

    embeddings = []
    with torch.no_grad():
        for batch in tqdm(loader, desc="Extracting Embeddings"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            cls_embeddings = outputs.last_hidden_state[:, 0, :]  # CLS token
            embeddings.append(cls_embeddings.cpu())
    return torch.cat(embeddings).numpy()

# ——— Generate and Save All Embeddings + Labels ———
all_embeddings = extract_embeddings(df["Statement"].tolist())
df_embeddings = pd.DataFrame(all_embeddings)
df_embeddings["label"] = df["Label"].values
df_embeddings.to_csv("TL_telugu_muril_embeddings.csv", index=False)

# ——— Train-Test Split ———
X = df_embeddings.drop("label", axis=1).values
y = df_embeddings["label"].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# ——— Feature Scaling ———
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# ——— Define Models and Parameter Grids ———
models_params = {
    "LogisticRegression": (
        LogisticRegression(max_iter=1000),
        {"C": [0.1, 1, 10]}
    ),
    "SVM": (
        SVC(),
        {"C": [0.1, 1, 10], "kernel": ["linear", "rbf"]}
    ),
    "RandomForest": (
        RandomForestClassifier(),
        {"n_estimators": [100, 200], "max_depth": [None, 10, 20]}
    ),
    "KNN": (
        KNeighborsClassifier(),
        {"n_neighbors": [3, 5, 7]}
    ),
    "XGBoost": (
        XGBClassifier(use_label_encoder=False, eval_metric="mlogloss"),
        {"n_estimators": [100, 200], "max_depth": [3, 5], "learning_rate": [0.05, 0.1]}
    )
}

# ——— Train + Evaluate Each Model ———
for name, (model, param_grid) in models_params.items():
    print(f"\n🔍 Running GridSearch for {name}...")
    grid = GridSearchCV(model, param_grid, cv=3, scoring="accuracy", n_jobs=-1)
    grid.fit(X_train, y_train)
    y_pred = grid.predict(X_test)

    print(f"\n✅ Best Parameters for {name}: {grid.best_params_}")
    print(f"📊 Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    print("📑 Classification Report:\n", classification_report(y_test, y_pred, target_names=["Negative", "Neutral", "Positive"]))
    print("🔢 Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Extracting Embeddings: 100%|██████████| 41/41 [00:02<00:00, 15.40it/s]



🔍 Running GridSearch for LogisticRegression...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 


✅ Best Parameters for LogisticRegression: {'C': 0.1}
📊 Accuracy: 0.6512
📑 Classification Report:
               precision    recall  f1-score   support

    Negative       0.57      0.64      0.60        85
     Neutral       0.70      0.71      0.70        85
    Positive       0.70      0.61      0.65        88

    accuracy                           0.65       258
   macro avg       0.66      0.65      0.65       258
weighted avg       0.66      0.65      0.65       258

🔢 Confusion Matrix:
 [[54 15 16]
 [18 60  7]
 [23 11 54]]

🔍 Running GridSearch for SVM...

✅ Best Parameters for SVM: {'C': 10, 'kernel': 'rbf'}
📊 Accuracy: 0.6860
📑 Classification Report:
               precision    recall  f1-score   support

    Negative       0.59      0.68      0.63        85
     Neutral       0.81      0.72      0.76        85
    Positive       0.68      0.66      0.67        88

    accuracy                           0.69       258
   macro avg       0.70      0.69      0.69       258
wei

/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [11:50:52] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [11:50:53] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [11:50:53] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [11:50:53] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [11:50:53] WARNING: /workspace/src/learner.cc:740: 
Parame


✅ Best Parameters for XGBoost: {'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 200}
📊 Accuracy: 0.6512
📑 Classification Report:
               precision    recall  f1-score   support

    Negative       0.57      0.59      0.58        85
     Neutral       0.78      0.71      0.74        85
    Positive       0.62      0.66      0.64        88

    accuracy                           0.65       258
   macro avg       0.66      0.65      0.65       258
weighted avg       0.66      0.65      0.65       258

🔢 Confusion Matrix:
 [[50 12 23]
 [13 60 12]
 [25  5 58]]
